<a href="https://colab.research.google.com/github/Chloe0704/Titanic/blob/master/HW1_Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier

In [0]:
from google.colab import files
uploaded = files.upload()

Saving train.csv to train.csv


In [0]:
import pandas as pd
df = pd.read_csv("train.csv",na_values='NULL')
df_test = pd.read_csv("test.csv",na_values='NULL')
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [0]:
df.drop_duplicates(inplace = True)
df_test.drop_duplicates(inplace = True)

In [0]:
df = df.drop(columns=['Cabin','Ticket','Name'])
df_test = df_test.drop(columns=['Cabin','Ticket','Name'])

In [0]:
df.info()

In [0]:
median_age = df['Age'].median()
median_age
median_age_t = df_test['Age'].median()
median_age_t
median_fare_t = df_test['Fare'].median()
median_fare_t


14.4542

In [0]:
df['Age'].fillna(median_age, inplace = True)
df_test['Age'].fillna(median_age_t, inplace = True)
df_test['Fare'].fillna(median_fare_t, inplace = True)

In [0]:
df["Embarked"] = df["Embarked"].fillna("S")
df_test["Embarked"] = df_test["Embarked"].fillna("S")

In [0]:
df_Emb = pd.get_dummies(df['Embarked'])
df = df.join(df_Emb)
df_Emb_t = pd.get_dummies(df_test['Embarked'])
df_test = df_test.join(df_Emb_t)

In [0]:
df_sex = pd.get_dummies(df['Sex'])
df = df.join(df_sex)
df_sex_t = pd.get_dummies(df_test['Sex'])
df_test = df_test.join(df_sex_t)

In [0]:
df = df.drop(columns=['Sex','Embarked'])
df_test = df_test.drop(columns=['Sex','Embarked'])


In [0]:
df_test.info()

In [0]:
target='Survived' #EXPECTED OUTPUT
IDcol= 'PassengerId'
x_columns = [x for x in df.columns if x not in [target,IDcol]]

X = df[x_columns]
y = df['Survived']



rf0 = RandomForestClassifier(n_estimators= 100, oob_score=True, random_state=10)
model0 = rf0.fit(X,y)
print(rf0.oob_score_)
y_predprob = rf0.predict_proba(X)[:,1]
print("AUC Score (Train): %f" % metrics.roc_auc_score(y, y_predprob))




0.8092031425364759
AUC Score (Train): 0.997020


In [0]:
from sklearn.model_selection import GridSearchCV


In [0]:
#1.n_estimators  (118)
param_test1 = {'n_estimators':list(range(100,130,2))}  
gsearch1 = GridSearchCV(estimator = RandomForestClassifier(min_samples_split=100,
                                  min_samples_leaf=20,max_depth=8,max_features='sqrt' ,oob_score = True ,n_jobs = -1,random_state=10), 
                       param_grid = param_test1, scoring='roc_auc',cv=5)
gsearch1.fit(X,y)
gsearch1.best_params_, gsearch1.best_score_

({'n_estimators': 118}, 0.8517660796224016)

In [0]:
#2.max_depth & min_samples_split      (7,2)     
param_test2 = {'max_depth':list(range(2,10,1)), 'min_samples_split':list(range(2,10,2))}
gsearch2 = GridSearchCV(estimator = RandomForestClassifier(n_estimators= 118, 
                                  min_samples_leaf=20,max_features='sqrt' ,oob_score=True, n_jobs = -1, random_state=10),
   param_grid = param_test2, scoring='roc_auc',iid=False, cv=5)
gsearch2.fit(X,y)
gsearch2.best_params_, gsearch2.best_score_



({'max_depth': 7, 'min_samples_split': 2}, 0.8576196862688216)

In [0]:
rf1 = RandomForestClassifier(n_estimators= 118, max_depth=7, min_samples_split=2,
                                  oob_score=True, random_state=10)
model1 = rf1.fit(X,y)
print(rf1.oob_score_)
y_predprob = rf1.predict_proba(X)[:,1]
print("AUC Score (Train): %f" % metrics.roc_auc_score(y, y_predprob))



0.8271604938271605
AUC Score (Train): 0.950010


In [0]:
rf1.feature_importances_

array([0.1192955 , 0.13982418, 0.04938071, 0.03812851, 0.18310323,
       0.01386301, 0.00783895, 0.01365733, 0.20303183, 0.23187676])

In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Age            891 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Fare           891 non-null float64
C              891 non-null uint8
Q              891 non-null uint8
S              891 non-null uint8
female         891 non-null uint8
male           891 non-null uint8
dtypes: float64(2), int64(5), uint8(5)
memory usage: 100.0 KB


In [0]:
#apply model to test data
PRED = rf0.predict(df_test[x_columns])

In [0]:
df_test['Survived'] = PRED
output = pd.DataFrame({'PassengerId':df_test['PassengerId'],'Survived':df_test['Survived']})
output.head()


,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,1
4,896,0


In [0]:
output.to_csv('Titanic.csv', index=False, sep=',')

In [0]:
from google.colab import files

files.download('Titanic.csv')

